In [1]:
import wrds
import time
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from datetime import datetime
from linearmodels.iv import IV2SLS
from linearmodels.iv import IVGMM

### 1. Load Data

In [2]:
df_regression = pd.read_csv("df_main_reg.csv")

In [3]:
df_rank = pd.read_csv("../Ressell_Rank/RR_data/df_rank_switch.csv") 
# '../' means move up one directory

#### -1) Create quarterly panel for each cusip

In [4]:
# Extract year from date
df_rank['year'] = pd.to_datetime(df_rank['date']).dt.year

# define switch indicator
switch_indicators = df_rank[['cusip', 'year', 
                             'switch_to_r2000', 
                             'switch_to_r1000', 
                             'switch_to_r3000']]

#### -2) Merge the switch instrument to each quarter

In [5]:
# Create a 'year' column from your quarterly date
df_regression['year'] = pd.to_datetime(df_regression['date']).dt.year

df_regression = df_regression.merge(switch_indicators, on=['cusip', 'year'], how='left')
for col in ['switch_to_r2000', 'switch_to_r1000', 'switch_to_r3000']:
    df_regression[col] = df_regression[col].fillna(0).astype(int)

In [6]:
df_regression.head(3)

,cusip,date,bas,tno,mktcap,price_ind,volume,illiq,volatility,synch,...,act_own,ins_own,auto_lag1,auto_lag2,auto_lag3,auto_lag4,year,switch_to_r2000,switch_to_r1000,switch_to_r3000
0,00030710,2014-12-31,0.084839,332.131683,178.587706,0.039206,7.001668,1.135563,0.277043,-3.648,...,0.084358,0.234046,0.041,0.169,0.203,0.078,2014,0,0,0
1,00030710,2015-03-31,0.051639,443.806176,274.286735,0.034110,9.355878,1.095236,0.317542,-1.841,...,0.131629,0.329042,0.223,0.120,0.025,0.305,2015,0,0,0
2,00030710,2015-06-30,0.046190,581.424544,465.762113,0.027190,12.664008,0.313499,0.249738,-2.326,...,0.114851,0.349641,0.106,0.062,0.202,0.156,2015,0,0,0


### 2. OLS Regression with IV

In [ ]:
# Define independent variables
market_quality_vars = ['bas', 'tno', 'illiq', 'volatility', 'synch', 'auto_lag1', 'auto_lag2', 'auto_lag3', 'auto_lag4']
Y = df_regression[market_quality_vars]

In [8]:
# Define X vars
index_var = ['ind_own']
control_vars = ['mktcap', 'price_ind', 'volume']

X_endog = df_regression[index_var]
X_exog = sm.add_constant(df_regression[control_vars]) #control vars are exogenous and add constant

In [9]:
# Define Instruments
instr_vars = ['switch_to_r2000', 'switch_to_r1000', 'switch_to_r3000']
Z_instr = df_regression[instr_vars]

#### Testing 2SLS Regression

In [29]:
# Example data (replace with your actual data)
data = pd.DataFrame({'y': [1, 2, 3, 4, 5], 'endog': [2, 3, 4, 5, 6], 'exog': [1, 2, 1, 2, 1], 'instruments': [3, 4, 5, 6, 7]})

# Define the model
formula = 'y ~ exog + [endog ~ instruments]'
mod = IV2SLS.from_formula(formula, data)
res = mod.fit()

# Access first-stage results
first_stage_results = res.first_stage
print(first_stage_results.diagnostics)

       rsquared  partial.rsquared  shea.rsquared       f.stat  f.pval   f.dist
endog  0.997436          0.987677       0.987677  2477.854767     0.0  chi2(1)


#### IV Results

In [12]:
# Collect results
iv_results = {}

for yvar in Y.columns:
    y = df_regression[yvar]

    model = IV2SLS(
        dependent=y,
        exog=X_exog,
        endog=X_endog,
        instruments=Z_instr
    ).fit(cov_type='robust')

    iv_results[yvar] = model


In [13]:
# Collect summary statistics in a table
summary_table = pd.DataFrame({
    var: {
        'coef_ind_own': model.params.get('ind_own', float('nan')),
        'pval_ind_own': model.pvalues.get('ind_own', float('nan')),
        'r2': model.rsquared,
        'first_stage_F': model.first_stage.diagnostics.get('f.stat', float('nan')).iloc[0]
    }
    for var, model in iv_results.items()
}).T


print(summary_table)

            coef_ind_own  pval_ind_own        r2  first_stage_F
bas             1.674338  0.000000e+00 -0.068685     647.760303
tno        -14141.565259  0.000000e+00 -0.342196     647.760303
illiq        -346.394626  5.847420e-01  0.019116     647.760303
volatility     -0.309622  8.817234e-03  0.157438     647.760303
synch          -3.783845  1.681509e-04 -0.047527     647.760303
auto_lag1       0.661470  5.329071e-15 -0.169085     647.760303
auto_lag2       0.231307  1.739217e-03 -0.024680     647.760303
auto_lag3       0.079578  2.578633e-01 -0.003048     647.760303
auto_lag4       0.040247  5.774556e-01 -0.001174     647.760303


In [80]:
model.first_stage.diagnostics

,rsquared,partial.rsquared,shea.rsquared,f.stat,f.pval,f.dist
ind_own,0.028917,0.001656,0.001656,647.760303,0.0,chi2(3)


In [ ]:
### 2. GMM(Generalized Method of Moments) Regression

In [16]:
gmm_results = {}

for yvar in Y.columns:
    y = df_regression[yvar]
    
    model = IVGMM(
        dependent=y,
        exog=X_exog,
        endog=X_endog,
        instruments=Z_instr
    )
    results = model.fit(cov_type='robust')
    gmm_results[yvar] = results
    
    print(f"Results for {yvar}")
    print(results.summary)

Results for bas
                          IV-GMM Estimation Summary                           
Dep. Variable:                    bas   R-squared:                     -0.0695
Estimator:                     IV-GMM   Adj. R-squared:                -0.0695
No. Observations:              258075   F-statistic:                    1397.9
Date:                Wed, Jun 11 2025   P-value (F-stat)                0.0000
Time:                        03:28:51   Distribution:                  chi2(4)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const         -0.0235     0.0122    -1.9280     0.0539     -0.0474      0.0004
mktcap      1.812e-07  6.227e-08    

In [17]:
summary_data = []

for yvar, model in gmm_results.items():
    summary_data.append({
        'yvar': yvar,
        'coef_ind_own': model.params.get('ind_own', float('nan')),
        'pval_ind_own': model.pvalues.get('ind_own', float('nan')),
        'r2': model.rsquared,
        'J-stat': model.j_stat.stat,
        'J-pval': model.j_stat.pval
    })

summary_table = pd.DataFrame(summary_data).set_index('yvar')
print(summary_table)

            coef_ind_own  pval_ind_own        r2      J-stat        J-pval
yvar                                                                      
bas             1.685538  0.000000e+00 -0.069468   45.625561  1.237472e-10
tno        -14327.242729  0.000000e+00 -0.358318   59.889704  9.892087e-14
illiq       -1507.502328  1.103029e-04  0.007286   50.889674  8.901213e-12
volatility      0.035907  7.576925e-01  0.154504  526.164688  0.000000e+00
synch          -3.059171  2.078996e-03 -0.024444   92.677804  0.000000e+00
auto_lag1       0.684537  4.440892e-16 -0.179953    7.601830  2.235031e-02
auto_lag2       0.229184  1.905037e-03 -0.024250    0.933321  6.270929e-01
auto_lag3       0.082761  2.388654e-01 -0.003288    3.024242  2.204420e-01
auto_lag4       0.042498  5.562597e-01 -0.001283    1.899917  3.867571e-01
